## Setting up Environment
Set up environment for querying

### 1. Importing packages

In [1]:
import collections
import csv
import operator
import os
import sys

from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import functions


### 2. Setting python path for data and setting data paths

In [2]:
sys.path.append('/Users/abuckfire/side-projects/arson')
import static_data.fire_codes.lookup_tables as codes

In [3]:
years_available = map(str, range(2009, 2015))
file_prefix = "nfirs_arson_"
arson_data_path = os.path.join("..", "static_data", "nfirs_arson")
fire_depts_data_path = os.path.join("..", "static_data", "nfirs_fire_depts")
pop_by_zipcode_csv = os.path.join("..", "static_data", "zipcodes", "pop_per_zip.csv")

### 3. Initializing Spark Context and reading data in

In [4]:
sqlContext = SQLContext(sc)

In [5]:
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

In [6]:
depts = spark.read.csv(fire_depts_data_path + "/nfirs_arson_" + "2009.csv", header=True).select(["state", "fdid", "zip"])
arson_df = spark.read.csv(arson_data_path + "/nfirs_arson_" + "2009.csv", header=True)

In [236]:
joined_on_zip = arson.join(depts, ['state','fdid'])

## Questions

### Question 1: What is the most common motivation for starting a fire?

In [113]:
df1 = arson_df.groupBy(["mot_facts1", "state"]).count().withColumnRenamed("mot_facts1", "motivation").withColumnRenamed("count", "count_1")
df2 = arson_df.groupBy(["mot_facts2", "state"]).count().withColumnRenamed("mot_facts2", "motivation").withColumnRenamed("count", "count_2")
df3 = arson_df.groupBy(["mot_facts3", "state"]).count().withColumnRenamed("mot_facts3", "motivation").withColumnRenamed("count", "count_3")

motives = df1.join(df2, ["motivation", "state"], "outer").join(df3, ["motivation", "state"], "outer").na.fill(0)

In [59]:
motives_per_state = motives.withColumn("count", sum(motives[col] for col in ["count_1", "count_2", "count_3"]))

NameError: name 'motives' is not defined

In [62]:
COUNT = "count"
STATE = "state"
UNKNOWNS = ["UU", "NN", None]

def lookup_code(candidate_list, column, codebook):
    candidates = [row for row in candidate_list if row[column] not in UNKNOWNS]
    if candidates:
        return codebook[max(candidates, key=operator.itemgetter(1))[0]]
    return None

def build_max_count_dicts(df, column, codebook, state_codes=STATES):
    results = collections.defaultdict()
    for state in state_codes:
        count_list = df[df.state==state].select(column, COUNT).collect()
        results[state] = lookup_code(count_list, column, codebook)

    overall_res_df = df.groupBy(column).agg(functions.sum(COUNT)).collect()
    results["overall"] = lookup_code(overall_res_df, column, codebook)
    return results

In [232]:
STATES = ["AK", "AL", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [233]:
def get_motives(df, column="motivation"):
    df1 = df.groupBy(["mot_facts1", STATE]).count().withColumnRenamed("mot_facts1", column).withColumnRenamed(COUNT, "count_1")
    df2 = df.groupBy(["mot_facts2", STATE]).count().withColumnRenamed("mot_facts2", column).withColumnRenamed(COUNT, "count_2")
    df3 = df.groupBy(["mot_facts3", STATE]).count().withColumnRenamed("mot_facts3", column).withColumnRenamed(COUNT, "count_3")

    all_motives = df1.join(df2, [column, STATE], "outer").join(df3, [column, STATE], "outer").na.fill(0)

    motives_per_state = all_motives.withColumn(COUNT, sum(motives[col] for col in ["count_1", "count_2", "count_3"]))
    return build_max_count_dicts(motives_per_state, column, codes.SUSPECTED_MOTIVATION_FACTORS, STATES)


### Question 2: What are the most common materials used for starting a fire?

In [214]:
container = arson_df.groupBy(["state", "devi_cont"]).count()
spark_type = arson_df.groupBy(["state", "devi_ignit"]).count()
gas_type = arson_df.groupBy(["state", "devi_fuel"]).count()

In [56]:
def get_means(df, column="devi_ignit"):
    df1 = df.groupBy([column, STATE]).count()
    return build_max_count_dicts(df1, column, codes.IGNITION_DELAY_DEVICE)


In [63]:
means = get_means(arson_df)
means

defaultdict(None,
            {'AK': 'Open flame source',
             'AL': 'Open flame source',
             'AR': 'Open flame source',
             'AZ': 'Open flame source',
             'CA': 'Open flame source',
             'CO': 'Open flame source',
             'CT': 'Wick or fuse',
             'DE': 'Wick or fuse',
             'FL': 'Open flame source',
             'GA': 'Open flame source',
             'HI': 'Open flame source',
             'IA': 'Wick or fuse',
             'ID': 'Open flame source',
             'IL': 'Open flame source',
             'IN': 'Open flame source',
             'KS': 'Open flame source',
             'KY': 'Open flame source',
             'LA': 'Open flame source',
             'MA': 'Open flame source',
             'MD': 'Wick or fuse',
             'ME': 'Open flame source',
             'MI': 'Open flame source',
             'MN': 'Open flame source',
             'MO': 'Wick or fuse',
             'MS': 'Open flame source',
       

### Question 3: What is the most common type of property burned?

In [266]:
property_ownership = arson_df.groupBy("prop_owner").count()
property_ownership.show()

+----------+-----+
|prop_owner|count|
+----------+-----+
|         7|    2|
|         3|  128|
|         0|  243|
|      null|47203|
|         5|   25|
|         1| 7757|
|         4|   77|
|         2|  500|
+----------+-----+



In [267]:
print("The primary property type of property affected by arson is: {}".format(
    get_max_value(property_ownership, "prop_owner", codes.PROPERTY_OWNERSHIP)))

The primary property type of property affected by arson is: private


In [ ]:
def lookup_code_2(candidate, column, codebook):
    if candidate or candidate not in UNKNOWNS:
        return codebook[max(candidate, key=operator.itemgetter(1))[0]]
    return None


def build_max_count_dicts_2(df, column, codebook, state_codes=STATES):
    results = collections.defaultdict()
    for state in state_codes:
        count_list = df[df.state==state].select(column, COUNT).collect()
        results[state] = lookup_code(count_list, column, codebook)

    overall_res = df.groupBy(column).agg(functions.sum(COUNT)).alias(COUNT)
    results["overall"] = lookup_code(overall_res, column, codebook)
    return results

### Question 4: Number of Arsons per Month

In [64]:
date = "inc_date"


In [65]:
def parse_date_by_month(date_string):
    month_lookup = {
        "1": "January",
        "2": "February",
        "3": "March",
        "4": "April",
        "5": "May",
        "6": "June",
        "7": "July",
        "8": "August",
        "9": "September",
        "10": "October",
        "11": "November",
        "12": "December"
    }
    return month_lookup[date_string[:2] if date_string[:2] in month_lookup else date_string[0]]

parse_date_by_month_udf = functions.udf(parse_date_by_month, "string")

In [66]:
date_as_month_df = arson_df.withColumn(date, parse_date_by_month_udf(arson_df[date]))


In [74]:
overall = date_as_month_df.groupBy(date).count().withColumn("state", functions.lit("overall"))

In [76]:
overall.show()

+---------+-----+-------+
| inc_date|count|  state|
+---------+-----+-------+
|     July| 6197|overall|
| November| 5841|overall|
| February| 4088|overall|
|  January|  325|overall|
|    March| 4836|overall|
|  October| 5433|overall|
|      May| 4797|overall|
|   August| 4760|overall|
|    April| 4821|overall|
|     June| 4883|overall|
| December| 5727|overall|
|September| 4227|overall|
+---------+-----+-------+



In [102]:
def get_arson_per_month(df, column="inc_date"):
    parse_date_by_month_udf = functions.udf(parse_date_by_month, "string")
    date_as_month_df = df.withColumn(column, parse_date_by_month_udf(df[column]))
    by_state = date_as_month_df.groupBy([date, STATE]).count()
    overall = date_as_month_df.groupBy(date).count().withColumn("state", functions.lit("overall")).select("inc_date", "state", "count")
    return overall.union(by_state)

In [103]:
l = get_arson_per_month(arson_df)

In [104]:
l.show()

+---------+-------+-----+
| inc_date|  state|count|
+---------+-------+-----+
|     July|overall| 6197|
| November|overall| 5841|
| February|overall| 4088|
|  January|overall|  325|
|    March|overall| 4836|
|  October|overall| 5433|
|      May|overall| 4797|
|   August|overall| 4760|
|    April|overall| 4821|
|     June|overall| 4883|
| December|overall| 5727|
|September|overall| 4227|
|     July|   null|    3|
|  January|     CO|    6|
| February|     NM|  102|
|      May|     NV|  262|
| December|     TX|  371|
|  January|     MN|    1|
| December|     NV|  300|
|  October|     ND|   10|
+---------+-------+-----+
only showing top 20 rows



In [105]:
from elasticsearch import Elasticsearch

In [106]:
es = Elasticsearch()

In [108]:
es.info(pretty=True)

{u'cluster_name': u'elasticsearch',
 u'cluster_uuid': u'W1tOguHzQxynPoRGAh4rtQ',
 u'name': u'dX-_z_A',
 u'tagline': u'You Know, for Search',
 u'version': {u'build_date': u'2017-03-23T03:31:50.652Z',
  u'build_hash': u'3adb13b',
  u'build_snapshot': False,
  u'lucene_version': u'6.4.1',
  u'number': u'5.3.0'}}

In [113]:
def create_index():
    index = {
        "settings": {
            "analysis": {
                "analyzer": {
                    # for the vector scoring plug-in configuration
                    "payload_analyzer": {
                        "type": "custom",
                        "tokenizer": "whitespace",
                        "filter": "delimited_payload_filter"
                    }
                }
            }
        },
        "mappings": {
            "motives": {
                "properties": {
                    "state": {
                        "type": "string"
                    },
                    "motive": {
                        "type": "string"
                    }
                }
            },
            "method": {
                "properties": {
                    "state": {
                        "type": "string"
                    },
                    "method": {
                        "type": "string"
                    }
                }                
            },
            "ownership": {
                "properties": {
                    "state": {
                        "type": "string"
                    },
                    "ownership": {
                        "type": "string"
                    }
                }
            },
            "monthly_counts": {
                "properties": {
                    "state": {
                        "type": "string"
                    },
                    "month": {
                        "type": "string"
                    },
                    "count": {
                        "type": "integer"
                    }
                }
            }
        }
    }
    es.indices.create(index="arson_facts", body=index)


In [114]:
create_index()

In [116]:
l.write.format("es").save("arson_facts/monthly_counts")

In [118]:
print("ES index count:  %d" % es.count(index="arson_facts", doc_type="monthly_counts")['count'])
print(es.search(index="arson_facts", doc_type="monthly_counts", q="*", size=3))


ES index count:  612
{u'hits': {u'hits': [{u'_score': 1.0, u'_type': u'monthly_counts', u'_id': u'AWZglakeJ9NbRKc0y1jr', u'_source': {u'count': 233, u'state': u'NV', u'inc_date': u'October'}, u'_index': u'arson_facts'}, {u'_score': 1.0, u'_type': u'monthly_counts', u'_id': u'AWZglaknJ9NbRKc0y1jw', u'_source': {u'count': 43, u'state': u'MN', u'inc_date': u'July'}, u'_index': u'arson_facts'}, {u'_score': 1.0, u'_type': u'monthly_counts', u'_id': u'AWZglaknJ9NbRKc0y1jy', u'_source': {u'count': 18, u'state': u'PA', u'inc_date': u'June'}, u'_index': u'arson_facts'}], u'total': 612, u'max_score': 1.0}, u'_shards': {u'successful': 5, u'failed': 0, u'total': 5}, u'took': 29, u'timed_out': False}


In [143]:
from pyspark.sql import Row

In [141]:
def convert_to_row(d):
    return Row(**collections.OrderedDict(sorted(d.items())))

def dict_to_df(json, field1, field2):
    items = [{field1: key, field2: value} for key, value in json.items()]
    return sc.parallelize(items).map(convert_to_row).toDF()

def ingest_facts_into_elastic(dataframe, index, doc_type):
    dataframe.write.format("es").save(index + "/" + doc_type)

In [142]:
ingest_facts_into_elastic(dict_to_df(means, "state", "method"), "arson_facts", "method")

In [144]:
print(es.search(index="arson_facts", doc_type="method", q="*", size=3))


{u'hits': {u'hits': [{u'_score': 1.0, u'_type': u'method', u'_id': u'AWZgrnNjJ9NbRKc0y1tF', u'_source': {u'state': u'VT'}, u'_index': u'arson_facts'}, {u'_score': 1.0, u'_type': u'method', u'_id': u'AWZgrnNjJ9NbRKc0y1tI', u'_source': {u'state': u'IN', u'method': u'Open flame source'}, u'_index': u'arson_facts'}, {u'_score': 1.0, u'_type': u'method', u'_id': u'AWZgrnNjJ9NbRKc0y1tM', u'_source': {u'state': u'ID', u'method': u'Open flame source'}, u'_index': u'arson_facts'}], u'total': 51, u'max_score': 1.0}, u'_shards': {u'successful': 5, u'failed': 0, u'total': 5}, u'took': 4, u'timed_out': False}


In [151]:
#ingest_facts_into_elastic(dict_to_df(motives, "state", "motivation"), "arson_facts", "motive")
#ingest_facts_into_elastic(dict_to_df(means, "state", "method"), "arson_facts", "method")
#ingest_facts_into_elastic(dict_to_df(ownership, "state", "ownership"), "arson_facts", "ownership")
#ingest_facts_into_elastic(l, "arson_facts", "monthly_counts")
es.index(index="arson_facts", doc_type="monthly_counts", body={"state": "x", "month": "october", "count": 10})


{u'_id': u'AWZgtbcZJ9NbRKc0y1t7',
 u'_index': u'arson_facts',
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'monthly_counts',
 u'_version': 1,
 u'created': True,
 u'result': u'created'}

In [152]:
es.search(index="arson_facts", doc_type="monthly_counts", q=5)

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'AWZgldjCJ9NbRKc0y1ry',
    u'_index': u'arson_facts',
    u'_score': 3.9400566,
    u'_source': {u'count': 5, u'inc_date': u'July', u'state': u'KY'},
    u'_type': u'monthly_counts'},
   {u'_id': u'AWZglb7pJ9NbRKc0y1nc',
    u'_index': u'arson_facts',
    u'_score': 3.9288168,
    u'_source': {u'count': 5, u'inc_date': u'December', u'state': u'WY'},
    u'_type': u'monthly_counts'},
   {u'_id': u'AWZgla3IJ9NbRKc0y1kj',
    u'_index': u'arson_facts',
    u'_score': 2.9635084,
    u'_source': {u'count': 5, u'inc_date': u'October', u'state': u'SD'},
    u'_type': u'monthly_counts'},
   {u'_id': u'AWZglbFiJ9NbRKc0y1lE',
    u'_index': u'arson_facts',
    u'_score': 2.9635084,
    u'_source': {u'count': 5, u'inc_date': u'February', u'state': u'UT'},
    u'_type': u'monthly_counts'},
   {u'_id': u'AWZgld2fJ9NbRKc0y1sH',
    u'_index': u'arson_facts',
    u'_score': 2.9635084,
    u'_source': {u'count'